In [19]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..','..'))
import config, utils

# Create a Spark session
spark = utils.create_spark_session("playerStats")

fielding_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'fielding_data.csv')
bowling_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'bowling_data.csv')
batting_data = utils.load_data(spark,config.PROCESSED_DATA_DIR, 'batting_data.csv')

fielding_data.show(5)

[2024-11-15T10:48:16.700+0530] {utils.py:12} INFO - Creating Spark session.
[2024-11-15T10:48:16.731+0530] {utils.py:30} INFO - Spark session created successfully.
[2024-11-15T10:48:16.733+0530] {utils.py:39} INFO - Loading data from fielding_data.csv.


[2024-11-15T10:48:31.691+0530] {utils.py:39} INFO - Loading data from bowling_data.csv.


[2024-11-15T10:48:32.413+0530] {utils.py:39} INFO - Loading data from batting_data.csv.
+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|    Player| Country| Season|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+----------+--------+-------+--------------+---------------+--------------+-------------+-------------+--------------+
|A Ahmadhel|Bulgaria|2019/20|             0|            0.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2020|             3|            3.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|2020/21|             4|            4.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2021|             6|            6.0|           0.0|          0.0|          0.0|           0.0|
|A Ahmadhel|Bulgaria|   2023|             9|            9.0|           0.0|    

In [20]:
print(fielding_data.count(), bowling_data.count(), batting_data.count())

14140 14140 14140


In [21]:
import config
from utils import load_data
players_data = load_data(spark, config.PROCESSED_DATA_DIR, 'match_players.csv').withColumnRenamed("country", "Country").withColumnRenamed("season", "Season")
players_data.show(5)    

[2024-11-15T10:48:34.139+0530] {utils.py:39} INFO - Loading data from match_players.csv.
+-------+-----------+---------+-------+--------+
|Country|     player|player_id| Season|match_id|
+-------+-----------+---------+-------+--------+
|England|   DN Wyatt| a139c379|2019/20| 1173063|
|England|TT Beaumont| b2664905|2019/20| 1173063|
|England|  NR Sciver| f3a18a0c|2019/20| 1173063|
|England|  HC Knight| 4ba0289e|2019/20| 1173063|
|England|  FC Wilson| a28132bd|2019/20| 1173063|
+-------+-----------+---------+-------+--------+
only showing top 5 rows



In [22]:
bowling_data.show(5)

+----------+--------+-------+--------------+---------------+------------------+---------------+---------------+---------------+
|    Player| Country| Season|Cumulative Mat|Cumulative Inns|  Cumulative Overs|Cumulative Runs|Cumulative Wkts|Cumulative Econ|
+----------+--------+-------+--------------+---------------+------------------+---------------+---------------+---------------+
|A Ahmadhel|Bulgaria|2019/20|             0|            0.0|               0.0|            0.0|            0.0|            0.0|
|A Ahmadhel|Bulgaria|   2020|             3|            3.0|              10.0|           75.0|            3.0|            7.5|
|A Ahmadhel|Bulgaria|2020/21|             4|            4.0|              12.0|           97.0|            4.0|           8.38|
|A Ahmadhel|Bulgaria|   2021|             6|            6.0|14.400000095367432|          124.0|            6.0|           8.96|
|A Ahmadhel|Bulgaria|   2023|             9|            8.0| 17.40000009536743|          155.0|         

In [23]:
players_data = players_data.join(fielding_data, ['Player', 'Country', 'Season'], 'inner').join(bowling_data, ['Player', 'Country', 'Season'], 'inner').join(batting_data, ['Player', 'Country', 'Season'], 'inner')
player_data = batting_data.join(bowling_data, on=['Player',"Country","Season"], how='inner').join(fielding_data, on=['Player',"Country","Season"], how='inner')\
                        .drop('Cumulative Mat','Cumulative Inns')
print(players_data.count())
players_data.show(5)

53489
+---------------+-------+------+---------+--------+--------------+---------------+--------------+-------------+-------------+--------------+--------------+---------------+-----------------+---------------+---------------+---------------+-------------+--------------+--------------+---------------+------+
|         player|Country|Season|player_id|match_id|Cumulative Mat|Cumulative Inns|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|Cumulative Mat|Cumulative Inns| Cumulative Overs|Cumulative Runs|Cumulative Wkts|Cumulative Econ|Cum Mat Total|Cum Inns Total|Cum Runs Total|Cum Batting Ave|Cum SR|
+---------------+-------+------+---------+--------+--------------+---------------+--------------+-------------+-------------+--------------+--------------+---------------+-----------------+---------------+---------------+---------------+-------------+--------------+--------------+---------------+------+
|    R Satheesan|Romania|  2022| 1125ebe4| 1310187|            17|           17

In [ ]:
"""Combine batting, bowling, and fielding statistics into a single dataset."""

import logging
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..','..'))
sys.path.append(os.path.join(os.getcwd(), '..'))
import config

import config
from utils import load_data, spark_save_data

def combine_data(spark):
    """Combine batting, bowling, and fielding statistics into a single dataset."""
    logging.info("Starting combine_data task.")

    try:
        batting_data = load_data(spark, config.PROCESSED_DATA_DIR, 'batting_data.csv')
        bowling_data = load_data(spark, config.PROCESSED_DATA_DIR, 'bowling_data.csv')
        fielding_data = load_data(spark, config.PROCESSED_DATA_DIR, 'fielding_data.csv')
        players_data = load_data(spark, config.PROCESSED_DATA_DIR, 'match_players.csv').withColumnRenamed("country", "Country").withColumnRenamed("season", "Season")

        batting_data = batting_data.join(players_data, ['Player', 'Country', 'Season'], 'inner')
        bowling_data = bowling_data.join(players_data, ['Player', 'Country', 'Season'], 'inner')
        fielding_data = fielding_data.join(players_data, ['Player', 'Country', 'Season'], 'inner')

        batting_data = batting_data.select(['player_id', 'Player', 'Country', "Season","Cum Mat Total", "Cum Runs Total", 'Cum SR']).sort("Player","Season")
        bowling_data = bowling_data.select(['player_id', 'Player', 'Country', "Season","Cumulative Mat", "Cumulative Inns", 'Cumulative Overs','Cumulative Runs','Cumulative Wkts','Cumulative Econ']).withColumnRenamed("Cumulative Runs","Cumulative Bowling Runs")
        fielding_data = fielding_data.select(['player_id', 'Player', 'Country', "Season","Cumulative Mat", "Cumulative Inns", 'Cumulative Dis','Cumulative Ct','Cumulative St','Cumulative D/I'])

        player_data = batting_data.join(bowling_data, on=['player_id','Player',"Country","Season"], how='inner').join(fielding_data, on=['player_id','Player',"Country","Season"], how='inner')\
                        .drop('Cumulative Mat','Cumulative Inns')
        print(player_data.count())
        print(player_data.show(5))
        spark_save_data(player_data, config.PROCESSED_DATA_DIR, 'player_stats.csv')
        logging.info("Data combining and saving completed successfully.")
    except Exception as e:
        logging.error(f"Error in combine_data task: {e}")
        raise
    finally:
        spark.stop()
        logging.info("Spark session stopped.")
        pass

24/11/15 10:54:51 WARN DataStreamer: Exception for BP-1744012875-127.0.1.1-1731345514466:blk_1073790793_49977
java.net.SocketTimeoutException: 65000 millis timeout while waiting for channel to be ready for read. ch : java.nio.channels.SocketChannel[connected local=/192.168.245.142:60464 remote=/192.168.245.142:9866]
	at org.apache.hadoop.net.SocketIOWithTimeout.doIO(SocketIOWithTimeout.java:163)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:161)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:131)
	at org.apache.hadoop.net.SocketInputStream.read(SocketInputStream.java:118)
	at java.base/java.io.FilterInputStream.read(FilterInputStream.java:83)
	at java.base/java.io.FilterInputStream.read(FilterInputStream.java:83)
	at org.apache.hadoop.hdfs.protocolPB.PBHelperClient.vintPrefixed(PBHelperClient.java:519)
	at org.apache.hadoop.hdfs.protocol.datatransfer.PipelineAck.readFields(PipelineAck.java:213)
	at org.apache.hadoop.hdfs.DataStreamer$R